In [258]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
from collections import defaultdict
import re
import sqlite3

Plan: walk through files
grab: title, ingredients(main), ingredients(detailed), old url, short description
convert dict to df and then write to SQLlite instance

TODO: 
get ingredients, instructions, 


In [248]:
pwd

'/Users/wolfiedieterich/Desktop/america_eats'

In [263]:
images = os.listdir()

In [265]:
images = [image[:-4] for image in images]

In [268]:
os.chdir("/Users/wolfiedieterich/Desktop/NYT_recipes")

In [10]:
with open("Classic Beef Brisket With Caramelized Onions.html") as fo:
    soup = bs(fo.read(), 'html.parser')

In [38]:
soup

<div class="grid-wrap">
<article class="card static noclickarea recipe-detail-card recipe-has-topnote recipe-has-media recipe-has-horizontal-media recipe-has-horizontal-media recipe-has-time-yield" data-analytics-category="Recipe Detail Full" data-id="1018922" data-seo-image-url="https://static01.nyt.com/images/2017/09/13/dining/13ROSHRECIPE/13ROSHRECIPE-videoSixteenByNineJumbo1600.jpg" data-type="recipe" data-url="/recipes/1018922-swiss-chard-and-lamb-torte-with-fennel-pomegranate-relish">
<header>
<div class="title-container">
<h1 class="recipe-title title name" data-name="Swiss Chard and Lamb Torte With Fennel-Pomegranate Relish" data-share-title="Swiss+Chard+and+Lamb+Torte+With+Fennel-Pomegranate+Relish+by+Joan+Nathan" data-url="/recipes/1018922-swiss-chard-and-lamb-torte-with-fennel-pomegranate-relish" itemprop="name">
          Swiss Chard and Lamb Torte With Fennel-Pomegranate Relish
        </h1>
</div>
<div class="recipe-subhead">
<h3 class="byline ">
<a class="author personal

In [13]:
def get_instructions(soup):
    l = [i.get_text().replace('Â', '') for i in soup.find(class_='recipe-steps').find_all('li')]
    try:
        l.extend([i.get_text().replace('Â', '') for i in soup.find(class_='recipe-notes').find_all('li')])
    except AttributeError as e:
        pass
    return l

In [14]:
def get_ingredients(soup): 
    ingredients = soup.find_all(itemprop='recipeIngredient')
    l = []
    for i in ingredients:
        l.append((i.find(class_='quantity').text.replace('Â',''), i.find(class_='ingredient-name').text))
    return l

In [15]:
get_ingredients(soup)

[('3', 'tablespoons kosher salt'),
 ('½', 'teaspoon black pepper'),
 ('1', 'teaspoon paprika (preferably Hungarian)'),
 ('', 'Pinch of cayenne'),
 ('5 to 7', 'pounds beef brisket, not too lean'),
 ('1', 'cup red or white wine'),
 ('12', 'cloves'),
 ('1', 'whole head of garlic'),
 ('3', 'bay leaves'),
 ('3', 'allspice berries'),
 ('6', 'large onions, peeled, and sliced crosswise 1/4-inch thick'),
 ('3', 'tablespoons olive oil'),
 ('', 'Parsley sprigs, for garnish'),
 ('¼', 'cup slivered scallions (optional)')]

In [27]:
def get_desc_tags(soup):
    l = []
    for i, child in enumerate(soup.find(class_='special-diets tag-block').children):
        contents = child.string.rstrip().lstrip()
        if re.search('[a-zA-Z]', child.string):
            l.append(child.string.rstrip().lstrip())
    return tuple(l)

In [253]:
file_list = [file for file in os.listdir() if '.html' in file]

In [278]:
os.chdir("/Users/wolfiedieterich/Desktop/NYT_recipes/templates")
file_list = [] 
for file in os.listdir():
    print(file)
    if '.html' in file and file[:-5] in images:
        file_list.append(file)

"21" Club Burger.html
"Gnocchi" of Hass Avocado with Apples and Shaved Walnuts.html
"Pudding" of Greens, Bread And Black-eyed Peas.html
'21' Club Hamburger.html
'Big Night' Timpano.html
'Twin Peaks' Cherry Pie.html
(Save the Polar) Bear Claws.html
.DS_Store
1-2-3-4 Cake.html
10-Minute Fruit Gratin.html
12-Fruit Compote.html
15-Minute Chocolate Cake.html
15-Minute Fried Herbed Chicken.html
1958: Eggnog.html
1968: Málaga Gazpacho.html
2006: Gazpacho With Cucumber Granita.html
2007: Paccheri With Caprese Lobster Salad.html
2007: Roquefort-and-Pear Eggnog.html
3-Bean Good Luck Salad With Cumin Vinaigrette.html
30-Vegetable Soup.html
45-Minute Roast Turkey.html
5 Ninth's Cubano.html
5-Ingredient Apple Pie.html
A Beer and a Shot.html
A Big Pot of Simmered Pintos.html
A Cheat's Bordelaise Sauce.html
A Cherry Tale.html
A Chicken in Every Teapot.html
A Country Syllabub.html
A Cream for All Seasons.html
A Crunchy Confection Chocolate Crickets.html
A Great Malted.html
A La Cote Saint-Jacques's G

Baked Cod Fish Fillets Florentine-Style.html
Baked Codfish With Spinach and Cheese Sauce.html
Baked Corn Casserole.html
Baked Custard.html
Baked Egg With Prosciutto and Tomato.html
Baked Eggplant With Ricotta, Mozzarella and Anchovy.html
Baked Eggs in Tomato Shells.html
Baked Eggs with Amaranth and Porcini.html
Baked Eggs With Onions And Cheese.html
Baked Farfalle With Eggplant.html
Baked Fennel.html
Baked Figs and Goat Cheese.html
Baked Figs.html
Baked Fillet of Sole With Tomato, Oregano And Hot Pepper.html
Baked Fish and Chips.html
Baked Fish With Sesame and Ginger.html
Baked Flounder and Eggs.html
Baked Flounder With Moroccan Spices.html
Baked French Toast With Oat Crumble Topping.html
Baked Fresh Ham.html
Baked Frittata With Green Peppers and Yogurt.html
Baked Frittata With Yogurt, Chard and Green Garlic.html
Baked Garbanzo Beans.html
Baked German Potato Salad.html
Baked Giant White Beans With Cabbage.html
Baked Glazed Pears.html
Baked Goat Cheese And Smoked Salmon Canapes.html
Bak

Buttermilk-Brown Sugar Waffles.html
Buttermilk-Marinated Lamb with Mustard Seeds and Savory.html
Buttermilk-Marinated Roast Chicken.html
Buttermilk-Marinated Wild Turkey With Peppery Milk Gravy.html
Butternut Mac-n-Cheese.html
Butternut Squash and Bacon Risotto.html
Butternut Squash And Chestnut Soup.html
Butternut Squash And Cider Soup.html
Butternut Squash and Green Curry Soup.html
Butternut Squash and Mushroom Wellington.html
Butternut Squash and Purple Potato Latkes.html
Butternut Squash and Roasted Apple Soup.html
Butternut Squash and Sage Latkes.html
Butternut Squash And Sage Risotto.html
Butternut Squash Bisque.html
Butternut Squash Cannoli.html
Butternut Squash Dip.html
Butternut Squash Kibbeh With Spiced Feta.html
Butternut Squash Oat Muffins With Candied Ginger.html
Butternut Squash Pie.html
Butternut Squash Polenta With Sausage and Onion.html
Butternut Squash Puree With Orange and Ginger.html
Butternut Squash Puree.html
Butternut Squash Ratatouille.html
Butternut Squash Rice

Comb Honey Cake.html
Comeback Sauce.html
Coming Up Roses.html
Commander's Palace Duck, Wild Mushroom And Andouille Filé Gumbo.html
Composed Salad With Green Olive and Garlic Vinaigrette.html
Compound Butter.html
Conch Chowder.html
Conch Steam.html
Concord Grape Mint Julep.html
Concord Pickled Lox.html
Condensed Milk Ice Cream.html
Condé Nast Cafeteria's Raspberry Pudding.html
Confectioners' Sugar Glaze.html
Confetti Green Bean Salad.html
Confit Chicken Salad.html
Confit De Canard.html
Confit Of Carrot And Cumin.html
Confit of Citrus Fruits.html
Confit Of Onions With Labneh Sauce.html
Congee With Napa Cabbage.html
Connemara Lamb Stew.html
Conventional Poached Eggs.html
Cooked Artichokes.html
Cooked Butterscotch Scotch Eggnog.html
Cooked Egg With Ibérico Ham.html
Cooked Grains Salad With Tomato Vinaigrette.html
Cooked Grapes With Cream.html
Cooked Tomatillo Salsa.html
Cooked Turkey Soup.html
Cooked Wild Rice.html
Cool Shrimp And Cucumber Slaw.html
Cool Soba Noodles With Sweet Soy Brot

Fish Stock For Bouillabaisse.html
Fish Stock.html
Fish Stuffed With Herbs, Walnuts and Pomegranate.html
Fish Tacos.html
Fish Veracruzana.html
Fish With Capers.html
Fish With Celery Root, Kale and Mushrooms.html
Fish With Cilantro And Garlic.html
Fish With Grilled Salsa Verde.html
Fish with honey and vinegar.html
Fish With Ratatouille.html
Fish with Shiitakes.html
Fish With Toasted Almonds.html
Fish With Tomato Salsa.html
Fish-Sauce Dip.html
Fish-Sauce-and-Black-Pepper Chicken Wings.html
Fishmonger’s Stew.html
Five Vegetables and 15-Grain Rice Pot.html
Five-Flavor Tea.html
Five-Hour Goat.html
Five-Minute Soup.html
Five-Peppercorn Fish Fillets.html
Five-Spice Chicken Livers.html
Five-Spice Crisp-Fried Squid.html
Five-Spice Duck Breast With Blackberries.html
Five-Spice Jasmine Rice With Portobello Mushrooms.html
Five-Spice Powder.html
Five-Spice Rum.html
Five-Spice Salmon With Sherry Vinegar and Roasted Chicken Sauce.html
Five-Spice Short Ribs.html
Five-Spice Shrimp And Pasta.html
Fizzy C

...

Pasta With Roasted Eggplant, Pepper and Garlic.html
Pasta With Roasted Garlic, Pancetta and Arugula.html
Pasta With Roasted Red Peppers and Goat Cheese.html
Pasta with Roasted Winter Squash and Ricotta Salata.html
Pasta With Salmon.html
Pasta With Salsa Crudo and Green Beans.html
Pasta With Sardines and Fennel.html
Pasta With Sardines, Bread Crumbs and Capers.html
Pasta With Scallops and Green Beans.html
Pasta With Seafood and Eggplant.html
Pasta With Seared Zucchini and Ricotta Salata.html
Pasta With Shell Beans and Tomatoes.html
Pasta With Shrimp and Vegetables.html
Pasta With Shrimp Ragù.html
Pasta With Smithfield Ham, Crab and Shiitakes in Creamy Garlic Sauce.html
Pasta With Smoked Mussels And Tomatoes.html
Pasta With Smoked Salmon and Yogurt.html
Pasta With Spicy Broccoli Rabe and Pine Nuts.html
Pasta With Spicy Sausages, Tomatoes, Rosemary and Olives.html
Pasta with Spicy Tomato Sauce.html
Pasta With Spinach And Blue Cheese.html
Pasta With Tomato, Zucchini And Cream Sauce.html
P

Red Cabbage Alsatian-Style.html
Red Cabbage and Apple Soup.html
Red Cabbage And Black Currant Salad.html
Red Cabbage and Black Rice, Greek Style.html
Red Cabbage Glazed With Maple Syrup.html
Red Cabbage Salad.html
Red Cabbage, Apples And Rice.html
Red Cabbage, Bacon And Stilton Filling.html
Red Cabbage, Carrot and Broccoli Stem Latkes With Caraway and Sesame.html
Red Chard, Potato and White Bean Ragout.html
Red Chilaquiles With Chicken.html
Red Chile Pork Brochettes.html
Red Coconut Rice Pudding With Mango.html
Red Flannel Potato Latkes.html
Red Hook Cocktail.html
Red Hot Ale.html
Red Kidney Bean Puree.html
Red Lemon.html
Red Lentil and Bulgur Kufteh.html
Red Lentil Dip.html
Red Lentil Kofta With Spinach.html
Red Lentil Soup With Lemon.html
Red Lentil Soup.html
Red Lentils and Chili Sauce With Quinoa.html
Red Onion Preserves.html
Red Onion Soup With Cheese Toasts.html
Red or Yellow Pepper Sauce.html
Red Pepper And Celery Root Salad.html
Red Pepper And Cucumber Salad.html
Red Pepper and

Sea Breeze.html
Sea Of Cortez Scallop Bisque.html
Sea Parsley Salad With Rice Wine Vinaigrette.html
Sea Scallop Carpaccio.html
Sea Scallop Ceviche.html
Sea Scallop Salad with Meyer Lemon and Pomegranate.html
Sea Scallop Sandwiches With Citrus Juices.html
Sea Scallops and Shrimp With Champagne Butter Sauce.html
Sea Scallops Cooked In Their Shells.html
Sea Scallops Grenobloise.html
Sea Scallops On Asparagus.html
Sea Scallops With Almond Sauce.html
Sea Scallops With Brown Butter, Capers and Lemon.html
Sea Scallops With Cider Glaze and Cauliflower Two Ways.html
Sea Scallops With Garlic and Tomatoes.html
Sea Scallops with Piquillo Peppers.html
Sea Scallops With Red Peppers and Tomatoes.html
Sea Scallops With Sugar Snap Peas.html
Sea Scallops With Sunchokes and Truffles.html
Sea Scallops With Sweet Peppers and Snow Peas.html
Sea Scallops With Sweet Red Peppers and Zucchini.html
Sea Scallops With Sweet Roasted Red Peppers.html
Sea Scallops With Tomatoes and Shallot Butter.html
Sea Trout Tarta

Stir-Fried Rainbow Peppers, Eggplant and Tofu.html
Stir-Fried Rice and Black Quinoa With Cabbage, Red Pepper and Greens.html
Stir-Fried Rice Noodles With Beets and Beet Greens.html
Stir-Fried Rice Stick Noodles With Bok Choy and Cherry Tomatoes.html
Stir-fried Rice With Amaranth or Red Chard and Thai Basil.html
Stir-Fried Sesame Shrimp and Spinach.html
Stir-Fried Shrimp With Black Beans.html
Stir-Fried Shrimp With Fennel, Lemon And Angel-Hair Noodles.html
Stir-Fried Shrimp With Snow Peas and Ginger.html
Stir-Fried Shrimp With Snow Peas and Red Peppers.html
Stir-Fried Shrimp With Spicy Greens.html
Stir-Fried Snow Peas With Soba.html
Stir-Fried Soba Noodles With Long Beans, Eggs and Cherry Tomatoes.html
Stir-Fried Soba Noodles With Turkey and Cabbage.html
Stir-Fried Spicy Asparagus.html
Stir-Fried Squid With Chinese Vegetables.html
Stir-fried Succotash With Edamame.html
Stir-Fried Sugar Snap Peas.html
Stir-Fried Sugar Snaps And Chanterelles.html
Stir-Fried Sweet Potatoes With Brown Butte

Veal Savarin.html
Veal Scaloppine With Hazelnuts and Balsamic Vinegar.html
Veal Shank With Shallots and Chanterelles.html
Veal Shanks With Carrots And Coriander.html
Veal Shanks With Dried Cranberries.html
Veal Shanks With Oriental Vegetables.html
Veal Shanks With Preserved Lemon.html
Veal Sliders With Wilted Cabbage.html
Veal Steak With Pesto (Hilaire).html
Veal Stew With Endive And Carrot.html
Veal Stew With Fennel and Tomatoes.html
Veal Stew With Pineapple And Tomato.html
Veal Stew With Sauteed Artichokes.html
Veal Stew With Spring Vegetables.html
Veal Stew With Tomato, Allspice and Fried Potatoes.html
Veal Stew.html
Veal Stock.html
Veal Tendrons in Cream and Tarragon Sauce.html
Veal With Artichoke, Black Olive And Mushroom Saute.html
Veal With Chanterelles.html
Veal With Dill Sauce.html
Veal With Fiddlehead and Shiitake Sauce.html
Veal With Tomatoes And Balsamic Vinegar.html
Veal With Tomatoes, Olives and Lemon.html
Veal, Pork And Chicken Granny's Meatloaf.html
Veal, Tuna and Caper

5905

In [267]:
'bob.html'[:-5]

'bob'

In [254]:
len(file_list)

16369

In [282]:
for file in file_list[:200]: print(file)

'Twin Peaks' Cherry Pie.html
(Save the Polar) Bear Claws.html
1958: Eggnog.html
1968: Málaga Gazpacho.html
2006: Gazpacho With Cucumber Granita.html
2007: Paccheri With Caprese Lobster Salad.html
2007: Roquefort-and-Pear Eggnog.html
3-Bean Good Luck Salad With Cumin Vinaigrette.html
5 Ninth's Cubano.html
5-Ingredient Apple Pie.html
A Beer and a Shot.html
A Big Pot of Simmered Pintos.html
A Meal in a Bowl: Salmon, Shiitakes and Peas.html
A Mess of Pork Chops With Dijon Dressing.html
A Plain Pizza Pie.html
A Very Updated Vegetable Chartreuse.html
Absinthe Frappé.html
Absurdly Easy Chocolate Fudge.html
Abuelo Peláez’s Frijoles Negros (Black Beans).html
Ad-Lib Turkey Cassoulet.html
Adobo-Fried Chicken.html
Afton Club Punch.html
Agridulce Royale.html
Agua Fresca de Jamaica.html
Ah, Sunflower!.html
Aioli Pan Bagnat or Stuffed Pita.html
Aioli.html
Ají Cuencano.html
Alabama Lemon ‘Cheese’ Cake.html
Alaskan Cod in Sweet and Sour Pepper Sauce.html
Albacore Roasted in a Bed of Lettuce.html
Al

In [289]:
d = defaultdict(dict)
for file in file_list[:500]:
    with open(file, 'r') as fo:
        try: 
            soup = bs(fo.read(), 'html.parser')
            d[file]['h1'] = soup.h1.get_text().lstrip().rstrip()
            d[file]['desc_tags'] = str(get_desc_tags(soup))
            d[file]['ingredients'] = str(get_ingredients(soup))
            d[file]['instructions'] = str(get_instructions(soup))
            d[file]['yield'] = soup.find(itemprop='recipeYield').get_text()
            d[file]['description'] = soup.find(itemprop='description').p.get_text()
        except Exception as e:
            continue


In [290]:
len(d)

500

In [291]:
df = pd.DataFrame.from_dict(d)

In [292]:
df = df.T

In [293]:
df = df.rename_axis('name')

In [311]:
df = df.dropna(axis=0, how='any')

In [312]:
len(df)

403

In [297]:
cd america_eats/

/Users/wolfiedieterich/Desktop/america_eats


In [142]:
pwd

'/Users/wolfiedieterich/Desktop/america_eats'

In [143]:
import sqlite3

In [313]:
db.close()

In [314]:
db = sqlite3.connect('am_v1.sqlite')

In [315]:
cursor = db.cursor()

In [316]:
cursor.execute("""drop table Recipes""")

In [317]:
db.commit()

In [318]:
cursor.execute("""create table Recipes (name text, desc_tags text, description text, h1 text, ingredients text,
                instructions text, yield text)""")

In [319]:
db.commit()

In [234]:
def make_dicts(cursor, row):
    return dict((cursor.description[idx][0], value)
                for idx, value in enumerate(row))
db.row_factory = make_dicts

In [309]:
df.isnull.count()

AttributeError: 'function' object has no attribute 'count'

In [320]:
df.to_sql('Recipes', db, if_exists='append')

In [303]:
response = cursor.execute("SELECT * FROM Recipes where h1 like '%Beef%'")

In [304]:
recipes = response.fetchall()

In [305]:
len(recipes)

25

In [226]:
recipe['desc_tags'] = eval(recipe['desc_tags'])

In [227]:
recipe['desc_tags']

('Jewish',
 'Middle Eastern',
 'Fennel',
 'Lamb',
 'Mint',
 'Pomegranate',
 'Rice',
 'Serrano Chile Pepper',
 'Swiss Chard')

In [184]:
type(recipes)

list

In [185]:
recipes

[{'desc_tags': "('Jewish', 'Allspice', 'Beef Brisket', 'Onion')",
  'description': 'This is a classic brisket recipe with no bells and whistles, just deep flavor, moist succulent meat and lots of caramelized onions. The only caveat: Buy a brisket that’s not too lean. You want it well-marbled with fat or the result will be dry, not juicy.',
  'h1': 'Classic Beef Brisket With Caramelized Onions',
  'ingredients': "[('3', 'tablespoons kosher salt'), ('½', 'teaspoon black pepper'), ('1', 'teaspoon paprika (preferably Hungarian)'), ('', 'Pinch of cayenne'), ('5 to 7', 'pounds beef brisket, not too lean'), ('1', 'cup red or white wine'), ('12', 'cloves'), ('1', 'whole head of garlic'), ('3', 'bay leaves'), ('3', 'allspice berries'), ('6', 'large onions, peeled, and sliced crosswise 1/4-inch thick'), ('3', 'tablespoons olive oil'), ('', 'Parsley sprigs, for garnish'), ('¼', 'cup slivered scallions (optional)')]",
  'instructions': "['In a small bowl, mix together salt, pepper, paprika and cay

In [172]:
df = pd.DataFrame(recipes)

array(["('Jewish', 'Allspice', 'Beef Brisket', 'Onion')"], dtype=object)

In [148]:
df1 = pd.DataFrame(response.fetchall())

In [149]:
df1

,0,1,2,3,4,5,6
0,Classic Beef Brisket With Caramelized Onions.html,"('Jewish', 'Allspice', 'Beef Brisket', 'Onion')",This is a classic brisket recipe with no bells...,Classic Beef Brisket With Caramelized Onions,"[('3', 'tablespoons kosher salt'), ('½', 'teas...","['In a small bowl, mix together salt, pepper, ...",6 to 8 servings
1,Fresh Fig Cake With Honey Cream-Cheese Frostin...,"('Cream Cheese', 'Fig', 'Honey', 'Jam')","This dense and deeply figgy cake, adapted from...",Fresh Fig Cake With Honey Cream-Cheese Frosting,"[('', 'Butter, for greasing the pan'), ('3', '...",['Make the cake: Heat oven to 325 degrees. But...,12 servings
2,Swiss Chard and Lamb Torte With Fennel-Pomegra...,"('Jewish', 'Middle Eastern', 'Fennel', 'Lamb',...",Festive dishes in Israel and throughout the Mi...,Swiss Chard and Lamb Torte With Fennel-Pomegra...,"[('2 ⅓', 'cups medium-grain or sushi rice (1 p...",['Prepare the torte: Place rice in a heat-proo...,10 to 12 servings


In [228]:
s = 'apple pie'

In [229]:
s = s.split()

In [230]:
s

['apple', 'pie']

In [ ]:
"Select * from recipes"

In [246]:
def make_query(s):
    query = "Select * from recipes where "
    for string in s:
        query += f"h1 like '%{string}%' and "
    query = query[:-4]
    return query
        

In [247]:
make_query(s)

"Select * from recipes where h1 like '%apple%' and h1 like '%pie%' "